In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import clip
from torch.nn import functional as F
import torch.nn as nn
from torchvision import transforms
from PIL import Image
train = True
classes = None
pictures= None

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("/home/geek/Workspace/BCI/Code/Decoding/BLIP/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("/home/geek/Workspace/BCI/Code/Decoding/BLIP/blip-image-captioning-base", torch_dtype=torch.float16).to("cuda")
def load_data():
    data_list = []
    label_list = []
    texts = []
    images = []
    blip_texts = []
    if train:
        text_directory = "/home/geek/Workspace/BCI/Data/THINGS/images_set/training_images"  
    else:
        text_directory = "/home/geek/Workspace/BCI/Data/THINGS/images_set/test_images"
    dirnames = [d for d in os.listdir(text_directory) if os.path.isdir(os.path.join(text_directory, d))]
    dirnames.sort()
    
    if classes is not None:
        dirnames = [dirnames[i] for i in classes]

    for dir in dirnames:
        try:
            idx = dir.index('_')
        except ValueError:
            print(f"Skipped: {dir} due to no '_' found.")
            continue
            
        new_description = f"{description}"
        texts.append(new_description)

    if train:
    else:
        img_directory ="/home/geek/Workspace/BCI/Data/THINGS/images_set/test_images"
    
    all_folders = [d for d in os.listdir(img_directory) if os.path.isdir(os.path.join(img_directory, d))]

    if classes is not None and pictures is not None:
        for i in range(len(classes)):
            class_idx = classes[i]
            pic_idx = pictures[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                if pic_idx < len(all_images):
                    images.append(os.path.join(folder_path, all_images[pic_idx]))                                                        

    elif classes is not None and pictures is None:
        for i in range(len(classes)):
            class_idx = classes[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                images.extend(os.path.join(folder_path, img) for img in all_images)
                
    elif classes is None:
        for idx, folder in enumerate(all_folders):
            folder_path = os.path.join(img_directory, folder)
            all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            all_images.sort()  
            images.extend(os.path.join(folder_path, img) for img in all_images)
            raw_image = Image.open(os.path.join(folder_path, all_images[0])).convert("RGB")
            # print(raw_image)
            # conditional image captioning
            text = [i.join(" ") for i in texts[idx].split("_")]
            inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)

            out = model.generate(**inputs)
            blip_text = processor.decode(out[0], skip_special_tokens=True)
            print(blip_text)
            blip_texts.append(blip_text)
    else:
        print("Error")
    return texts, blip_texts, images
texts, blip_texts, images = load_data()
# images
blip_texts

/home/geek/miniconda3/envs/BCI/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a small pig walking across a dirt covered ground
a wooden cabinet with glass doors and drawers
a person playing an accordion on a white table
a nut on a wooden table
a window air conditioner sitting on a window sie
a purple couch in a room with a white wall
a propor pump with a hose attached to it
a car with a bag in the passenger seat
a green boat with a motor on the shore
a small airplane flying in the sky
an open book with pictures of people on it
a crocodile and a crocodile in the water
almonds are a great source for many foods
a plant in a pot with a small plant in the background
a group of alda alpaca alda alpaca alpac alpac alpac
the altar in the church
a roll of foil on a wooden table
a yellow stone in the sky
a red and yellow ambulance parked in front of a house
an old radio sitting on top of a wooden table
a anchor on the waterfront
a person's foot with a small patch on it
a close up of a person's foot with a green and pink flower on it
a small ant ant ant ant ant ant ant ant

['a small pig walking across a dirt covered ground',
 'a wooden cabinet with glass doors and drawers',
 'a person playing an accordion on a white table',
 'a nut on a wooden table',
 'a window air conditioner sitting on a window sie',
 'a purple couch in a room with a white wall',
 'a propor pump with a hose attached to it',
 'a car with a bag in the passenger seat',
 'a green boat with a motor on the shore',
 'a small airplane flying in the sky',
 'an open book with pictures of people on it',
 'a crocodile and a crocodile in the water',
 'almonds are a great source for many foods',
 'a plant in a pot with a small plant in the background',
 'a group of alda alpaca alda alpaca alpac alpac alpac',
 'the altar in the church',
 'a roll of foil on a wooden table',
 'a yellow stone in the sky',
 'a red and yellow ambulance parked in front of a house',
 'an old radio sitting on top of a wooden table',
 'a anchor on the waterfront',
 "a person's foot with a small patch on it",
 "a close up of 

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import clip
from torch.nn import functional as F
import torch.nn as nn
from torchvision import transforms
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
vlmodel, preprocess = clip.load("ViT-B/32", device=device)


def Textencoder(text):   
    text_inputs = torch.cat([clip.tokenize(t) for t in text]).to(device)

    with torch.no_grad():
        text_features = vlmodel.encode_text(text_inputs)
    
    text_features = F.normalize(text_features, dim=-1).detach()

    return text_features
    
def ImageEncoder(images):
    image_features_list = []
    
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        image_inputs = torch.stack([preprocess(Image.open(img).convert("RGB")) for img in batch_images]).to(device)

        with torch.no_grad():
            batch_image_features = vlmodel.encode_image(image_inputs)
            batch_image_features /= batch_image_features.norm(dim=-1, keepdim=True)

        image_features_list.append(batch_image_features)

    image_features = torch.cat(image_features_list, dim=0)
    
    return image_features

In [6]:

classes = None
train = False
if classes is None and pictures is None:
    # Try to load the saved features if they exist
    features_filename = os.path.join('/home/geek/Workspace/BCI/Data/THINGS/CLIP', 'new_features_train.pt') if train else os.path.join('/home/geek/Workspace/BCI/Data/THINGS/CLIP', 'new_features_test.pt')
    
    text_features = Textencoder(blip_texts)
    img_features = ImageEncoder(images)
    torch.save({
        'new_text_features': text_features,
        'new_img_features': img_features,
    }, features_filename)
else:
    text_features = Textencoder(blip_texts)
    img_features = ImageEncoder(images)